<a href="https://colab.research.google.com/github/sarahasan17/NLP-Techniques-in-Spam-Classification/blob/main/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [4]:
data = pd.read_csv('spam.csv', encoding='latin-1')


In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
data['title_wo_punct']=data['v2'].apply(lambda x: remove_punctuation(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...


In [7]:
import re

In [8]:
def tokenize(text):
    split=re.split("\W+",text)
    return split
data['title_wo_punct_split']=data['title_wo_punct'].apply(lambda x: tokenize(x.lower()))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stopword = nltk.corpus.stopwords.words('english')

In [11]:
def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text
data['title_wo_punct_split_wo_stopwords'] = data['title_wo_punct_split'].apply(lambda x: remove_stopwords(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."


In [12]:
ps=nltk.PorterStemmer()
def stemming(tokenized_text):
  text=[ps.stem(word) for word in tokenized_text]
  return text
data['title_wo_punct_split_wo_stemming']=data['title_wo_punct_split'].apply(lambda x: stemming(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, i, dont, think, he, goe, to, usf, he, li..."


In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
  text=[wn.lemmatize(word) for word in tokenized_text]
  return text
data['text_wo_lemmatizer']=data['title_wo_punct_split_wo_stemming'].apply(lambda x: lemmatizing(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, until, jurong, point, crazi, avail, onli,...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, so, earli, hor, u, c, alreadi, t...","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, i, dont, think, he, goe, to, usf, he, li...","[nah, i, dont, think, he, goe, to, usf, he, li..."


In [15]:
X=data['v1']
y=data['text_wo_lemmatizer']
print(len(X)," ",len(y))

5572   5572


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [22]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score